**CNN-based Veterinary Animal Disease Detection**




**1.Problem Statement**

Animals can suffer from various diseases that are hard to identify visually by owners.
This project builds a CNN-based system to detect animal diseases from images and provide advice
for treatment or precaution.

**2. Objectives**
- Detect diseases in animal images using CNN.
- Provide treatment suggestions based on predicted disease.
- Evaluate model performance using metrics like accuracy, F1-score.
- Help owners take timely action for animal care.


**3. Dataset Description**
- 6 classes: Demodicosis, Dermatitis, Healthy, Hypersensitivity, Ringworm, Fungal_infections
- Images split into training   (~70-80%),  validation (~10-15%), and test (~10-15%)
- Folder structure example:
dataset/
train/
validation/
test/


**4. Solution Approach**
- Preprocess images: resize to 128x128, normalize pixel values.
- Train CNN with Conv2D, MaxPooling2D, BatchNormalization, Dropout.
- Use softmax output for 6 classes.
- Map predictions to suggestion dictionary for owner advice.
- Evaluate using classification report and confusion matrix.


**5. CNN Code**
# See attached code at the end for training and prediction in Colab.
(Training, saving, loading model, prediction with advice.)
6. Results
- Accuracy on validation: 0.67
- Weighted F1-score: 0.66
- Best performing classes: Healthy, Demodicosis, Ringworm
- Weak classes: Hypersensitivity, Fungal_infections
- Sample prediction: 'Image: 1000010494.jpg -> Predicted: Fungal_infections -> Suggestion: Consult
a vet for antifungal treatment.'

**7. Conclusion**
- The CNN-based system can detect animal diseases and provide actionable advice.
- Accuracy is good but can improve with more data, augmentation, and fine-tuning.
- Ready for deployment in a simple web/mobile interface for animal owners.

In [ ]:
import zipfile

with zipfile.ZipFile("/content/archive (1).zip", "r") as zip_ref:
    zip_ref.extractall("/content/")


In [ ]:
import zipfile

zip_path = "/content/archive (1).zip"  # change name if needed
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("/content/")
print("✅ Extracted successfully!")


In [ ]:
# ==============================
# 🐾 Veterinary Disease Detection using CNN
# Author: Akhil
# ==============================

# 1️⃣ Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os

# 2️⃣ Paths to dataset (make sure your dataset is extracted in /content)
train_path = "/content/content/dataset/train"
test_path = "/content/content/dataset/valid"

# 3️⃣ Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 4️⃣ Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(6, activation='softmax')  # 5 classes
])

# 5️⃣ Compile
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# 6️⃣ Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20
)

# 7️⃣ Plot accuracy and loss
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

# 8️⃣ Save the trained model
model.save("/content/vet_cnn_model.h5")
print("✅ Model saved as vet_cnn_model.h5")


In [ ]:
import os

test_dir = "/content/content/dataset/valid"  # 👈 change to your actual test folder path

# 1️⃣ Check if folder exists
if not os.path.exists(test_dir):
    print("❌ Test folder not found! Check your path:", test_dir)
else:
    print("✅ Test folder found.")
    print("📸 Total files:", len(os.listdir(test_dir)))

    # 2️⃣ Print first few filenames
    for i, f in enumerate(os.listdir(test_dir)[:10]):
        print(f"🔹", f)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# =========================
# 1️⃣ Paths and model
# =========================
val_dir = "/content/content/dataset/test"  # Folder with subfolders for each class
model_path = "/content/vet_cnn_model.h5"  # Your trained model
model = load_model(model_path)


suggestions = {
    "demodicosis": {
        "advice": "Visit a veterinary hospital immediately.",
        "description": "Demodicosis (mange) is caused by mites. It needs medicated baths and antiparasitic treatment."
    },
    "Dermatitis": {
        "advice": "Consult a vet for allergy testing or antibiotics.",
        "description": "Dermatitis can result from allergies, infections, or poor hygiene. Keep the animal clean and avoid irritants."
    },
    "Healthy": {
        "advice": "No treatment needed.",
        "description": "Your animal looks healthy! Maintain a good diet and hygiene."
    },
    "Hypersensitivity": {
        "advice": "Consult a vet if symptoms persist.",
        "description": "It may be due to an allergic reaction. Avoid suspected allergens and monitor the animal closely."
    },
    "ringworm": {
        "advice": "Visit a vet; avoid close contact.",
        "description": "Ringworm is a fungal infection that spreads to humans and other animals. Requires antifungal treatment."
    },
    "Fungal_infections": {   # ✅ Add this key
        "advice": "Consult a vet for antifungal treatment.",
        "description": "Fungal infections can spread easily. Keep the animal clean and use antifungal medication as prescribed by a vet."
    }
}



# =========================
# 3️⃣ Class labels
# =========================
class_labels = sorted(os.listdir(val_dir))  # sorted to maintain consistent order
print("Classes found:", class_labels)

# =========================
# 4️⃣ Store predictions for metrics
# =========================
y_true = []
y_pred = []


        # =========================
# 5️⃣ Loop through validation folder
# =========================
count = 0
for class_folder in class_labels:
    folder_path = os.path.join(val_dir, class_folder)
    if not os.path.isdir(folder_path):
        continue
    for img_name in os.listdir(folder_path):
        if not img_name.lower().endswith(('.png','.jpg','.jpeg')):
            continue
        count += 1
        img_path = os.path.join(folder_path, img_name)

        # Load & preprocess
        img = image.load_img(img_path, target_size=(128,128))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)/255.0

        # Predict
        pred = model.predict(img_array, verbose=0)
        pred_class = class_labels[np.argmax(pred)]
        confidence = np.max(pred)

        # Store for metrics
        y_true.append(class_folder)
        y_pred.append(pred_class)

        # Display image + prediction + suggestion
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"{pred_class} ({confidence*100:.1f}%)")
        plt.show()

        # Print prediction + advice
        print(f"Image: {img_name}")
        print(f"Predicted Class: {pred_class}")
        print(f"Confidence: {confidence*100:.2f}%")
        print(f"Suggestion: {suggestions[pred_class]['advice']}")
        print(f"Description: {suggestions[pred_class]['description']}")
        print("-"*60)


print(f"\n✅ Processed {count} images.")



In [ ]:
# =========================
# 6️⃣ Metrics
# =========================
print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred, labels=class_labels)
plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_labels, yticklabels=class_labels, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()